In [1]:
from ai import dprint
from marubatsu import Marubatsu
from copy import deepcopy
from functools import wraps
from time import perf_counter
from random import choice

def ai_by_mmscore(eval_func):
    @wraps(eval_func)
    def wrapper(mb_orig, debug=False, *args, rand=True, share_tt=True,
                analyze=False, calc_score=False, **kwargs):       
        if calc_score:
            score, count = eval_func(mb_orig, debug, *args, **kwargs)
            return score
        
        starttime = perf_counter()
        dprint(debug, "Start ai_by_mmscore")
        dprint(debug, mb_orig)
        legal_moves = mb_orig.calc_legal_moves()
        dprint(debug, "legal_moves", legal_moves)
        maxnode = mb_orig.turn == Marubatsu.CIRCLE
        best_score = float("-inf") if maxnode else float("inf")
        best_moves = []
        tt = {} if share_tt else None
        totalcount = 0
        if analyze:
            score_by_move = {}
        for move in legal_moves:
            dprint(debug, "=" * 20)
            dprint(debug, "move", move)
            mb = deepcopy(mb_orig)
            x, y = move
            mb.move(x, y)
            dprint(debug, mb)
            score, count = eval_func(mb, debug, tt=tt, *args, **kwargs)
            totalcount += count
            dprint(debug, "score", score, "best score", best_score)
            if analyze:
                score_by_move[move] = score
            
            if (maxnode and best_score < score) or (not maxnode and best_score > score):
                best_score = score
                best_moves = [move]
                dprint(debug, "UPDATE")
                dprint(debug, "  best score", best_score)
                dprint(debug, "  best moves", best_moves)
            elif best_score == score:
                best_moves.append(move)
                dprint(debug, "APPEND")
                dprint(debug, "  best moves", best_moves)

        dprint(debug, "=" * 20)
        dprint(debug, "Finished")
        dprint(debug, "best score", best_score)
        dprint(debug, "best moves", best_moves)
        bestmove = choice(best_moves) if rand else best_moves[0]
        if analyze:
            return {
                "candidate": best_moves,
                "score_by_move": score_by_move,
                "tt": tt,
                "time": perf_counter() - starttime,
                "bestmove": bestmove,
                "score": best_score,
                "count": totalcount,
            }
        else:
            return bestmove
        
    return wrapper

In [2]:
@ai_by_mmscore
def ai_abs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1, eval_func=None,
               eval_params={}, use_tt=False, tt=None):           
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = max(score, ab_search(mb, depth + 1, tt, alpha, beta))
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = min(score, ab_search(mb, depth + 1, tt, alpha, beta))
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
        from util import calc_same_boardtexts

        if use_tt:
            boardtxtlist = calc_same_boardtexts(mborig)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt in boardtxtlist:
                tt[boardtxt] = (lower_bound, upper_bound)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [3]:
from ai import ai14s

mb = Marubatsu()
eval_params = { "minimax": True }
print(ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params))

(2, 2)


In [4]:
print(ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params, calc_score=True))

0.0


In [5]:
from pprint import pprint

pprint(ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params, analyze=True))

{'bestmove': (1, 0),
 'candidate': [(0, 0),
               (1, 0),
               (2, 0),
               (0, 1),
               (1, 1),
               (2, 1),
               (0, 2),
               (1, 2),
               (2, 2)],
 'count': 30709,
 'score': 0.0,
 'score_by_move': {(0, 0): 0.0,
                   (0, 1): 0.0,
                   (0, 2): 0.0,
                   (1, 0): 0.0,
                   (1, 1): 0.0,
                   (1, 2): 0.0,
                   (2, 0): 0.0,
                   (2, 1): 0.0,
                   (2, 2): 0.0},
 'time': 1.5907108001410961,
 'tt': {}}


In [6]:
pprint(ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params, 
                  use_tt=True,  analyze=True))

{'bestmove': (1, 1),
 'candidate': [(0, 0),
               (1, 0),
               (2, 0),
               (0, 1),
               (1, 1),
               (2, 1),
               (0, 2),
               (1, 2),
               (2, 2)],
 'count': 1175,
 'score': 0.0,
 'score_by_move': {(0, 0): 0.0,
                   (0, 1): 0.0,
                   (0, 2): 0.0,
                   (1, 0): 0.0,
                   (1, 1): 0.0,
                   (1, 2): 0.0,
                   (2, 0): 0.0,
                   (2, 1): 0.0,
                   (2, 2): 0.0},
 'time': 0.16863269917666912,
 'tt': {'........o': (0.0, 0.0),
        '.......o.': (0.0, 0.0),
        '.......ox': (0.0, 0.0),
        '.......xo': (300, 300),
        '......o..': (0.0, 0.0),
        '......o.x': (300, inf),
        '......oox': (-300, -300),
        '......ox.': (300, 300),
        '......oxo': (0.0, 0.0),
        '......x.o': (300, inf),
        '......xo.': (0.0, 0.0),
        '......xoo': (-300, -300),
        '.....o...': 

In [7]:
@ai_by_mmscore
def ai_abs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1, eval_func=None,
               eval_params={}, use_tt=False, tt=None):           
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound, _ = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                if abscore > score:
                    bestmove = (x, y)
                score = max(score, abscore)
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                if abscore < score:
                    bestmove = (x, y)
                score = min(score, abscore)
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
        from util import calc_same_boardtexts

        if use_tt:
            boardtxtlist = calc_same_boardtexts(mborig)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt in boardtxtlist:
                tt[boardtxt] = (lower_bound, upper_bound, bestmove)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [8]:
result = ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params, 
                    use_tt=True, analyze=True)
tt = result["tt"]
pprint(tt)

{'........o': (0.0, 0.0, (1, 1)),
 '.......o.': (0.0, 0.0, (0, 0)),
 '.......ox': (0.0, 0.0, (0, 1)),
 '.......xo': (300, 300, (0, 1)),
 '......o..': (0.0, 0.0, (1, 1)),
 '......o.x': (300, inf, (0, 1)),
 '......oox': (-300, -300, (2, 1)),
 '......ox.': (300, 300, (0, 1)),
 '......oxo': (0.0, 0.0, (1, 1)),
 '......x.o': (300, inf, (0, 1)),
 '......xo.': (0.0, 0.0, (0, 1)),
 '......xoo': (-300, -300, (2, 1)),
 '.....o...': (0.0, 0.0, (0, 0)),
 '.....o..x': (0.0, 0.0, (0, 1)),
 '.....o.ox': (0.0, 0.0, (1, 1)),
 '.....o.x.': (300, inf, (0, 0)),
 '.....o.xo': (300, 300, (2, 0)),
 '.....oo.x': (-inf, 0.0, (1, 1)),
 '.....oox.': (-inf, 300, (2, 0)),
 '.....ooxx': (300, 300, (1, 1)),
 '.....ox..': (300, inf, (0, 0)),
 '.....ox.o': (300, inf, (1, 0)),
 '.....oxo.': (-inf, 0.0, (1, 1)),
 '.....oxox': (300, 300, (1, 1)),
 '.....oxxo': (300, 300, (1, 1)),
 '.....x..o': (300, 300, (0, 1)),
 '.....x.o.': (300, inf, (0, 0)),
 '.....x.oo': (300, 300, (2, 0)),
 '.....xo..': (300, inf, (2, 0)),
 '.....

In [9]:
from tree import Mbtree_GUI

Mbtree_GUI()

In [10]:
print(tt[mb.board_to_str()])

KeyError: '.........'

In [11]:
bestmove = result["bestmove"]
PV = []
while mb.status == Marubatsu.PLAYING:
    PV.append(bestmove)
    print(bestmove)
    x, y = bestmove
    if mb.board[x][y] != Marubatsu.EMPTY:
        print("そのマスには着手済みです")
        break
    mb.move(x, y)
    print(mb)
    boardtxt = mb.board_to_str()
    if boardtxt in tt:
        _, _, bestmove = tt[boardtxt]
    elif mb.status != Marubatsu.PLAYING:
        print("置換表にデータが登録されていません")
        break
print(PV)

(0, 2)
Turn x
...
...
O..

(1, 1)
Turn o
...
.X.
o..

(1, 0)
Turn x
.O.
.x.
o..

(1, 0)
そのマスには着手済みです
[(0, 2), (1, 1), (1, 0), (1, 0)]
